<h2>Appendix 3 - Cleaning Text Fields</h2>

Our data includes four text fields that require pre-processing prior to analysis: User, Location, Description, and Text. The steps set out in plain text below are described in greater detail in the Pre-Processing section of our study.

**Steps for all fields**
<ul>
<li>Capitalise text to simplify future token matching</li>
</ul>

**Steps for Location and User**
<ul>
<li>Remove all punctuation</li>
</ul>

**Steps for Description and Text**
<ul>
<li>Remove URLs in truncated tweets</li>
<li>Remove all punctuation except "#" and "@", used to identify hashtags and user mentions</li>
<li>Identify hashtags and mentions, stripping them from the original fields and preserving them in new columns</li>
</ul>

**Steps for Text only**
<ul>
<li>Split camelcase hashtags into composite words for use in sentiment analysis</li>
<li>Remove emoji, preserving them in a new column</li>
</ul>

The unicode value ranges used to identify emoji were taken from https://apps.timwhitlock.info/emoji/tables/unicode. The system is clearly imperfect as a number of emoji are not successfully identified. As such, they were not used for sentiment analysis

In [1]:
import pandas as pd
import numpy as np
import re
import string

In [2]:
tweet_data = pd.read_excel("sotu_final_corpus.xlsx")

In [3]:
# Create empty columns to populate later
tweet_data["description_hashtags"] = ""
tweet_data["description_mentions"] = ""
tweet_data["text_emoji"] = ""
tweet_data["text_hashtags"] = ""
tweet_data["text_hashtags_split"] = ""
tweet_data["text_mentions"] = ""

In [4]:
# Write regex pattern to remove all punctuation
remove = string.punctuation
remove = remove + "“”‘’"
punct_pattern = r"[{}]".format(remove)

# Write second regex pattern to remove all punctuation except '#' and '@'
remove = remove.replace("#", "")
remove = remove.replace("@", "")
special_punct_pattern = r"[{}]".format(remove)

In [5]:
for i in range(len(tweet_data.index)):
    
    # Capitalise and remove puncutation from user field
    tweet_data.at[i,"user"] = re.sub(punct_pattern, " ", tweet_data.at[i,"user"]).upper()
    
    # Capitalise and remove punctuation from location field, avoiding empty fields
    location = tweet_data.at[i,"location"]
    if type(location) == str:
        tweet_data.at[i,"location"] = re.sub(punct_pattern, " ", location).upper()    
    
    # Clean description field
    description = tweet_data.at[i,"description"]
    if type(description) == str:
        description = re.sub(r"http\S+", "", description)  # Remove URLs
        description = re.sub(special_punct_pattern, " ", description)  # Remove punctuation
        
        # Strip hashtags and mentions into separate column
        tweet_data.at[i,"description_hashtags"] = str.join(" ",[word.strip("#") for word in description.split() if word.startswith("#")]).upper()
        tweet_data.at[i,"description_mentions"] = str.join(" ",[word.strip("@") for word in description.split() if word.startswith("@")]).upper()
        tweet_data.at[i,"description"] = str.join(" ",[word for word in description.split() if (not word.startswith("#")) and (not word.startswith("@"))]).upper()
        
    # Clean text field
    text = tweet_data.at[i,"text"]
    if type(text) == str:
        text = re.sub(r"http\S+", "", text) # Remove URLs
        text = re.sub(special_punct_pattern, " ", text) # Remove punctuation
        
        # Strip emoji into separate column
        tweet_data.at[i, "text_emoji"] = str.join(" ", [word for word in text.split() if (('\U0001f601' <= word <= '\U0001f64f') or ('\U00002702' <= word <= '\U000027b0') or ('\U0001f680' <= word <= '\U0001f6c5') or ('\U0001f170' <= word <= '\U0001f251') or ('\U0001f300' <= word <= '\U0001f5ff') or word == '\U0000236a')])
        text = str.join(" ", [word for word in text.split() if word not in (tweet_data.at[i, "text_emoji"])])
        
        # Strip hashtags and mentions into separate columns
        tweet_data.at[i,"text_hashtags"] = str.join(" ",[word.strip("#") for word in text.split() if word.startswith("#")])
        tweet_data.at[i,"text_mentions"] = str.join(" ",[word.strip("@") for word in text.split() if word.startswith("@")]).upper()
        tweet_data.at[i,"text"] = str.join(" ",[word for word in text.split() if (not word.startswith("#")) and (not word.startswith("@"))]).upper()
        
        # Split camel case hashtags
        hashtags = tweet_data.at[i,"text_hashtags"]
        tweet_data.at[i,"text_hashtags_split"] = re.sub(r'((?<=[a-z])[A-Z]|(?<!\A)[A-Z](?=[a-z]))', r' \1', hashtags).upper()
        tweet_data.at[i,"text_hashtags"] = tweet_data.at[i,"text_hashtags"].upper()    

In [7]:
# Save data set
writer = pd.ExcelWriter('sotu_processed.xlsx')
tweet_data.to_excel(writer,'Sheet1')
writer.save()